# Squashed Entanglement
$$
 \newcommand{\ul}[1]{\underline{#1}}
 \newcommand{\rvalp}[0]{{\ul{\alpha}}}
\newcommand{\alp}[0]{{\alpha}}
 \newcommand{\rvx}[0]{{\ul{x}}}
\newcommand{\rvy}[0]{{\ul{y}}}
$$

The purpose of this notebook is to show how to use entanglish to
calculate the squashed entanglement of a mixed state (either pure or not pure).

Consider a bipartite system
consisting of two parts labelled
by the random variables $\rvx$ and $\rvy$,
and
described by a density matrix $\rho_{\rvx, \rvy}$.
The squashed entanglement of  such a system
is defined as

$$
E_{\rvx, \rvy}(\rho_{\rvx, \rvy}) =
\frac{1}{2}
\min S(\rvx : \rvy|\rvalp)
\;.
$$
The min()---or infimum()
if one wishes to be more mathematically
precise--is
over all density matrices $\rho_{\rvx, \rvy,\rvalp}$
such that ${\rm tr}_\rvalp \; \rho_{\rvx, \rvy,\rvalp}=
\rho_{\rvx, \rvy}$ with $\rho_{\rvx, \rvy}$ held fixed.
If $\rho_{\rvx, \rvy}$ is a pure state, then
$E_{\rvx, \rvy} = S(\rvx) = S(\rvy)$.
Entanglish-Original-Ref discusses other interesting
properties of squashed entanglement

Entanglish-Original-Ref also describes the algo
used by Entanglish to calculate squashed entanglement. The
algorithm is recursive. 
The number of recursive steps 
can be chosen by the user and is called num_ab_steps (ab stands
for Arimoto-Blahut).
Another parameter of the algorithm is num_hidden_states, which is the number of possible 
$\rvalp$ values.

 
**Entanglish-Original-Ref**
* "A New  Algorithm for Calculating
Squashed Entanglement and a Python Implementation Thereof", by R.R.Tucci

First change your working directory to the entanglish directory in your computer, and add its path to the path environment variable.

In [1]:
import os
import sys
print(os.getcwd())
os.chdir('../../')
print(os.getcwd())
sys.path.insert(0,os.getcwd())

C:\Users\rrtuc\Desktop\backed-up\python-projects\entanglish\entanglish\jupyter_notebooks
C:\Users\rrtuc\Desktop\backed-up\python-projects\entanglish


In [2]:
from entanglish.SymNupState import *
from entanglish.SquashedEnt import *

## pure states (symmetrized n-up states)
Next we construct a symmetrized n-up pure state.
Then we compare the Arimoto-Blahut algo entanglement value to the known 
entanglement value, for various possible
bi-partitions of the set of row axes. As expected, they are equal.

recursion_init is a str that specifies how to
initiate the recursion for Kxy_a. There are currently 2 options for recursion_init, 'eigen' and 'equi-diag'.
With 'eigen', the recursion converges instantly for pure states. 

In [3]:
num_bits = 4
num_up = 1
dm1 = DenMat(1 << num_bits, tuple([2]*num_bits))
st = SymNupState(num_up, num_bits)
st_vec = st.get_st_vec()
dm1.set_arr_from_st_vec(st_vec)

num_hidden_states = dm1.num_rows
num_ab_steps = 5
print('num_hidden_states=', num_hidden_states)
print('num_ab_steps=', num_ab_steps)
ecase = SquashedEnt(dm1, num_hidden_states, num_ab_steps, 
                    recursion_init='eigen', verbose=True)
print('entang_023: algo value, known value\n',
      ecase.get_entang({0, 2, 3}),
      st.get_known_entang(3))
print('entang_02: algo value, known value\n',
      ecase.get_entang({0, 2}),
      st.get_known_entang(2))
print('entang_1: algo value, known value\n',
      ecase.get_entang({1}),
      st.get_known_entang(1))


num_hidden_states= 16
num_ab_steps= 5

--ab step= 0 , entang= 0.5623351446188081 , err= 0.0
--ab step= 1 , entang= 0.5623351446188095 , err= 0.0
--ab step= 2 , entang= 0.5623351446188088 , err= 0.0
--ab step= 3 , entang= 0.5623351446188082 , err= 0.0
--ab step= 4 , entang= 0.5623351446188076 , err= 0.0
entang_023: algo value, known value
 0.5623351446188076 0.5623351446188083

--ab step= 0 , entang= 0.6931471805599445 , err= 0.0
--ab step= 1 , entang= 0.6931471805599454 , err= 0.0
--ab step= 2 , entang= 0.6931471805599455 , err= 0.0
--ab step= 3 , entang= 0.6931471805599453 , err= 0.0
--ab step= 4 , entang= 0.6931471805599453 , err= 0.0
entang_02: algo value, known value
 0.6931471805599453 0.6931471805599453

--ab step= 0 , entang= 0.5623351446188078 , err= 0.0
--ab step= 1 , entang= 0.5623351446188086 , err= 0.0
--ab step= 2 , entang= 0.5623351446188083 , err= 0.0
--ab step= 3 , entang= 0.562335144618808 , err= 0.0
--ab step= 4 , entang= 0.5623351446188088 , err= 0.0
entang_1: algo v

## random density matrices
Next we consider 2 random density matrices (actually,
only their eigenvectors are random. Their eigenvalues are specified by the user.)
For each of those 2 density matrices, we calculate the Arimoto-Blahut algo entanglement value, for various possible
bi-partitions of the set of row axes.

In [4]:
np.random.seed(123)
dm = DenMat(8, (2, 2, 2))
evas_of_dm_list = [
    np.array([.07, .03, .25, .15, .3, .1, .06, .04])
    , np.array([.05, .05, .2, .2, .3, .1, .06, .04])
    ]
num_hidden_states = dm.num_rows
num_ab_steps = 60
print('num_hidden_states=', num_hidden_states)
print('num_ab_steps=', num_ab_steps)
for evas_of_dm in evas_of_dm_list:
    evas_of_dm /= np.sum(evas_of_dm)
    print('***************new dm')
    print('evas_of_dm\n', evas_of_dm)
    dm.set_arr_to_rand_den_mat(evas_of_dm)
    ecase = SquashedEnt(dm, num_hidden_states, num_ab_steps, 
                        recursion_init='equi-diag', verbose=True)
    print('ent_02_1=', ecase.get_entang({0, 2}))

num_hidden_states= 8
num_ab_steps= 60
***************new dm
evas_of_dm
 [0.07 0.03 0.25 0.15 0.3  0.1  0.06 0.04]

--ab step= 0 , entang= -0.222195240564792 , err= 6.586416260333297
--ab step= 1 , entang= 0.09273051639350684 , err= 0.08447919852863123
--ab step= 2 , entang= 0.09388249188783987 , err= 0.07322574301785395
--ab step= 3 , entang= 0.09494899744548752 , err= 0.08601696578502414
--ab step= 4 , entang= 0.09644814919326758 , err= 0.10779731920368173
--ab step= 5 , entang= 0.098568568107986 , err= 0.13600359618849292
--ab step= 6 , entang= 0.10110422216481936 , err= 0.16623863353306473
--ab step= 7 , entang= 0.10294457900864112 , err= 0.1899139223588225
--ab step= 8 , entang= 0.10203367951099376 , err= 0.1961787657542885
--ab step= 9 , entang= 0.09682737354516532 , err= 0.1800251993926981
--ab step= 10 , entang= 0.08808393083019428 , err= 0.14777598759062474
--ab step= 11 , entang= 0.07833480765230022 , err= 0.11119725864975502
--ab step= 12 , entang= 0.06984874237867086 , err= 

--ab step= 56 , entang= 0.060746064916270485 , err= 0.011124871894400472
--ab step= 57 , entang= 0.06070906389280914 , err= 0.01092220000675072
--ab step= 58 , entang= 0.060672324664410276 , err= 0.010722512063993883
--ab step= 59 , entang= 0.06063583667435374 , err= 0.010525876297867885
ent_02_1= 0.06063583667435374
